In [2]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
import torch
import pandas as pd
import re
from collections import defaultdict

# Load your essay file
path = "/home/semipro321/OneDrive/School/McGill/Research Projects/Essay/"
df = pd.read_csv(path + "essays_corrected_200.csv")
model_name = "lorenzozan/ME2-BERT"


In [ ]:

#number 6 is a good essay and 14
text = df['corrected'][14]
max_length = 256



mf_mapping = {'CH':'CARE/HARM','FC':'FAIRNESS/CHEATING', 'LB':'LOYALTY/BETRAYAL', 'AS':'AUTHORITY/SUBVERSION', 'PD': 'PURITY/DEGRADATION'}


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)


encoded_input = tokenizer(
    text,
    max_length=max_length,  
    padding="max_length",  
    truncation=True,  
    return_tensors="pt", 
)

input_ids = encoded_input["input_ids"]
attention_mask = encoded_input["attention_mask"]

model.eval()  
with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)



In [57]:
for i, tt in enumerate(text):
    print(tt)
    for mf, score in output[i].items():
        print(f'{mf_mapping[mf]} : {score}')
        
    print()

I
CARE/HARM : 0.88686
FAIRNESS/CHEATING : 0.06897
LOYALTY/BETRAYAL : 0.79819
AUTHORITY/SUBVERSION : 0.00429
PURITY/DEGRADATION : 0.04236

 


IndexError: list index out of range

In [34]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd

# Load your essay file
path = "/home/semipro321/OneDrive/School/McGill/Research Projects/Essay/Data/essay11/Processed/"
df = pd.read_csv(path + "essays_corrected.csv")


model_name = "lorenzozan/ME2-BERT"
max_length = 256

mf_mapping = {
    'CH':'CARE/HARM',
    'FC':'FAIRNESS/CHEATING',
    'LB':'LOYALTY/BETRAYAL',
    'AS':'AUTHORITY/SUBVERSION',
    'PD':'PURITY/DEGRADATION'
}
codes = list(mf_mapping.keys())  # ['CH','FC','LB','AS','PD']

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
model.eval()

all_scores = []
with torch.no_grad():
    for text in df['corrected']:
        enc = tokenizer(
            text,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        out = model(**enc)  
        # unwrap if it's a list/tuple
        if isinstance(out, (list, tuple)):
            scores_tensor = out[0]
        else:
            scores_tensor = out
        # now scores_tensor is a Tensor of shape [1,5] or [5]
        values = scores_tensor.squeeze().tolist()
        # build dict
        mapped = { mf_mapping[codes[i]]: values[i] for i in range(len(codes)) }
        all_scores.append(mapped)

df_scores = pd.DataFrame(all_scores)
df = pd.concat([df.reset_index(drop=True), df_scores], axis=1)
#df.to_csv(path + "essays_with_mfscores.csv", index=False)

In [35]:
df.to_csv(path + "essays_with_mfscores.csv", index=False)


In [ ]:
df["AUTHORITY/SUBVERSION"].describe()

count    10509.000000
mean         0.074843
std          0.188835
min          0.000023
25%          0.000502
50%          0.001789
75%          0.013164
max          0.980539
Name: AUTHORITY/SUBVERSION, dtype: float64

In [41]:
df[df["AUTHORITY/SUBVERSION"] == df["AUTHORITY/SUBVERSION"].max()]

,Text,ID,Words,cleaned_text,corrected,CARE/HARM,FAIRNESS/CHEATING,LOYALTY/BETRAYAL,AUTHORITY/SUBVERSION,PURITY/DEGRADATION
409,ID: N28214N----------------------I would first...,N28214N,114.0,I would first try to get a house and then I wo...,I would first try to get a house and then I wo...,0.988878,0.942886,0.971639,0.980539,0.532303
